In [1]:
import sys
import os

sys.path.append("/home/vib9/src/SL-Net/superlayer/voxel")

from train import train
import argparse
import logging
import os
import sys

import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split

sys.path.append("../../")
sys.path.append("../../scripts/")
import superlayer.utils

from scripts import eval_net, train_net, get_args

from superlayer.models import AESuperNet, AEnet, cvpr2018_net
from superlayer.utils import BrainD, dice_coeff, one_hot, plot_img_array, plot_side_by_side

SyntaxError: non-default argument follows default argument (voxelnets.py, line 51)

In [ ]:
dir_img = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/vols/'
dir_mask = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/asegs/'

dir_train = '/home/vib9/src/SL-Net/jupyter/partitions/train.txt'
dir_val = '/home/vib9/src/SL-Net/jupyter/partitions/val.txt'

dir_checkpoint_1 = 'checkpoints_1/'
dir_checkpoint_2 = 'checkpoints_2/'

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
args = get_args()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')

In [ ]:
target_label_numbers = [0,2,3,4,10,16,17,28,31,41,42,43,49,53,63]
val_percent = 0.1
batch_size = args.batchsize
lr = args.lr
img_scale = args.scale
writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')
enc_nf = [64, 64, 64, 64]
dec_nf = [64, 64, 64, 64]

In [ ]:
dir_i = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/vols/'
atlas_file = '/home/vib9/src/voxelmorph/data/atlas_norm.npz'
# Prepare the vm1 or vm2 model and send to device
nf_enc = [64, 64, 64, 64]
nf_dec = [64, 64, 64, 64, 64, 64, 64]
atlas_vol = np.load(atlas_file)['vol'][np.newaxis, ..., np.newaxis][:,:,:,100,:]
vol_size = atlas_vol.shape[1:-1]

model = cvpr2018_net(vol_size, nf_enc, nf_dec)
    
train(model,
      gpu='0',
      data_dir=dir_i,
      val_dir=dir_val,
      atlas_file=atlas_file,
      lr=1e-4,
      n_iter=150000,
      data_loss='mse',
      model='vm2',
      reg_param=0.01, 
      batch_size=1,
      n_save_iter=500,
      model_dir='/home/vib9/src/voxelmorph/pytorch/models/')

In [ ]:
net1 = AESuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=64, depth=4, ignore_last=True)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net1.to(device=device)
train_scores1, val_scores1, train_var_1, val_var_1 = train_net(net=net1,
                                                                epochs=5,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val,
                                                                segment=False)

In [ ]:
np.save("/home/vib9/src/SL-Net/superlayer/models/superblocks/TRUE_AE_64.npy",net1.W.cpu().data)

In [ ]:
net2 = AEnet(input_ch=1, out_ch=15, use_bn=True, enc_nf=enc_nf, dec_nf=enc_nf, ignore_last=True)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net2.to(device=device)
train_scores2, val_scores2, train_var_2, val_var_2 = train_net(net=net2,
                                                                epochs=5,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val,
                                                                segment=False)

In [ ]:
import math

train_path = '/home/vib9/src/SL-Net/jupyter/partitions/train.txt'
val_path = '/home/vib9/src/SL-Net/jupyter/partitions/val.txt'

train = BrainD(dir_img, dir_mask, id_file=train_path, label_numbers=target_label_numbers)
val = BrainD(dir_img, dir_mask, id_file=val_path, label_numbers=target_label_numbers)

train_loader = DataLoader(train, batch_size=2, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val, batch_size=2, shuffle=False, num_workers=8, pin_memory=True, drop_last=True)

n_val = len(val)
n_train = len(train)
dataset_size = n_val + n_train

# Get the first batch
train_batch = next(iter(train_loader))
val_batch = next(iter(val_loader))

inputs_train = train_batch['image']
inputs_val = val_batch['image']


In [ ]:
net1.eval() 
inputs = inputs_train.to(device=device, dtype=torch.float32)

# Predict
pred = net1(inputs)
# The loss functions include the sigmoid function.
pred = pred.data.cpu().numpy()

input_images_rgb = [x.squeeze(0) for x in inputs.cpu().numpy()]
pred_rgb = [x.squeeze(0) for x in pred]

plot_side_by_side([input_images_rgb, pred_rgb])

In [ ]:
net2.eval() 
inputs = inputs_train.to(device=device, dtype=torch.float32)

# Predict
pred = net2(inputs)
# The loss functions include the sigmoid function.
pred = pred.data.cpu().numpy()

input_images_rgb = [x.squeeze(0) for x in inputs.cpu().numpy()]
pred_rgb = [x.squeeze(0) for x in pred]

plot_side_by_side([input_images_rgb, pred_rgb])